# Momentum, Volatility, and Volume Factors in U.S. Stock Returns

**ISYE 4031 Final Project**  
*Regression & Forecasting, Georgia Tech*

## Project Overview

This notebook analyzes the relationship between **momentum**, **volatility**, and **volume** factors in U.S. stock returns using S&P 500 data.

### Research Questions:
1. Do momentum indicators significantly predict future stock returns?
2. How does volatility clustering affect return predictability? 
3. Is trading volume a reliable indicator of price direction?

---

In [2]:
import yfinance as yf
import pandas as pd
from pandas_datareader import data as pdr
import datetime as dt
import numpy as np
from bs4 import BeautifulSoup
import requests, re
import ta

---
### S&P 500 Stock List
We start by scraping the current S&P 500 stock list from a reliable financial data source.

**Data Source**: [Stock Analysis - S&P 500](https://stockanalysis.com/list/sp-500-stocks/)

**Key Information Collected**:
- Stock symbols (tickers)
- Market capitalization



In [3]:
url = 'https://stockanalysis.com/list/sp-500-stocks/'
resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html.parser')

# Find the table and extract headers
table = soup.find('table', class_='symbol-table svelte-1ro3niy')
headers = [th.get_text(strip=True) for th in table.find('tr').find_all('th')]

# Extract all row data
stocks_data = []
for row in table.find_all('tr')[1:]:  # Skip header row
    row_data = [cell.get_text(strip=True) for cell in row.find_all('td')]
    stocks_data.append(row_data)

# Create DataFrame and set No. column as index
sp500_df = pd.DataFrame(stocks_data, columns=headers)
sp500_df = sp500_df.set_index('No.')

print("\nFirst 10 rows:")
print(sp500_df.head(10)[['Symbol', 'Market Cap']])


First 10 rows:
    Symbol Market Cap
No.                  
1     NVDA      4.68T
2     AAPL      4.07T
3     MSFT      3.78T
4     GOOG      3.52T
5    GOOGL      3.45T
6     AMZN      2.63T
7     AVGO      1.68T
8     META      1.55T
9     TSLA      1.43T
10   BRK.B      1.07T


---

### Stock Selection & Date Range Setup

**Stock Selection Process:**
- Extract first 50 companies from S&P 500 list for analysis
- Focus on established companies for reliable historical data

**Analysis Time Period:**
- **Start Date**: January 1, 2021
- **End Date**: December 27, 2024
- **Duration**: 4 years of market data
- **Purpose**: Capture post-pandemic market trends and recovery patterns

> **Note**: Using a subset of the top 50 stocks for computational efficiency and financial significance.



In [4]:
stocks = sp500_df.head(15)['Symbol'].tolist()

# Filter out all tickers that contain dots (they often cause yfinance issues)
stocks = [ticker for ticker in stocks if '.' not in ticker]

stocks.sort()
startDate = dt.date(2021, 1, 4)
endDate = dt.date(2024, 12, 27)

print(f"Selected stocks: {stocks}")
print(f"Total stocks: {len(stocks)}")

Selected stocks: ['AAPL', 'AMZN', 'AVGO', 'GOOG', 'GOOGL', 'JPM', 'LLY', 'META', 'MSFT', 'NVDA', 'ORCL', 'TSLA', 'V', 'WMT']
Total stocks: 14


---

### Weekly Returns and Factor Calculation

**Objective**: Calculate weekly log returns and prepare data structure for technical indicator analysis.

**Key Metrics**:
- **Weekly Log Returns**: `ln(Close/Open) × 100` for price movement analysis
- **ROC** Rate of Change indicator for momentum analysis.
- **RVOL** Relative Volume indicator for volume analysis.
- **BBW** Bollinger Band width indicator for volatility analysis.

**Data Structure**:
- Multi-level columns for organized factor storage
- Separate columns for each technical indicator per stock
- Week numbering for time series tracking

In [ ]:
# Download the data with extended lookback for indicator calculations
try:
    # Extend start date by 4 months for proper technical indicator calculations (need 50+ trading days)
    extended_start = startDate - pd.DateOffset(months=4)
    
    # Download daily data with extended period for more precise indicator calculations
    daily_download = yf.download(
        tickers = stocks,
        start = extended_start,
        end = endDate,
        actions = False, threads = True, auto_adjust = True, rounding = True,
        group_by = 'tickers', 
        interval = '1d'  # Daily data for daily-based indicators
    )
    
    # Extract OHLCV data
    daily_open = daily_download.xs('Open', level=1, axis=1)
    daily_close = daily_download.xs('Close', level=1, axis=1)
    daily_high = daily_download.xs('High', level=1, axis=1)
    daily_low = daily_download.xs('Low', level=1, axis=1)
    daily_volume = daily_download.xs('Volume', level=1, axis=1)
    
    # Convert daily to weekly data (Friday close) for analysis
    weekly_open = daily_open.resample('W-FRI').first()
    weekly_close = daily_close.resample('W-FRI').last()
    weekly_high = daily_high.resample('W-FRI').max()
    weekly_low = daily_low.resample('W-FRI').min()
    weekly_volume = daily_volume.resample('W-FRI').sum()
    
    # Filter to analysis period (Jan 4, 2021 onwards)
    analysis_start = pd.Timestamp('2021-01-04')
    analysis_mask = weekly_close.index >= analysis_start
    
    # Get analysis period data
    analysis_close = weekly_close[analysis_mask]
    analysis_open = weekly_open[analysis_mask]
    analysis_high = weekly_high[analysis_mask]
    analysis_low = weekly_low[analysis_mask]
    analysis_volume = weekly_volume[analysis_mask]
    
    # Calculate weekly log returns
    log_returns = (np.log(analysis_close / analysis_open) * 100)
    
    # Create MultiIndex DataFrame
    columns = []
    for ticker in stocks:
        columns.extend([(ticker, 'Log_Return_%'), (ticker, 'ROC_36d_lag1'), (ticker, 'RVOL_50d_lag1'), (ticker, 'BBW_36d_lag1')])
    multi_columns = pd.MultiIndex.from_tuples(columns, names=['Ticker', 'Metric'])
    weekly_data = pd.DataFrame(index=analysis_close.index, columns=multi_columns)
    
    # Calculate indicators for each stock using daily data
    for ticker in stocks:
        if ticker in daily_close.columns:
            # Get full daily time series for calculations (including lookback period)
            ticker_daily_close = daily_close[ticker].dropna()
            ticker_daily_volume = daily_volume[ticker].dropna()
            
            if len(ticker_daily_close) > 60:  # Need sufficient daily data for indicators
                
                # 1. Log Returns (current week - already calculated from weekly data)
                weekly_data[(ticker, 'Log_Return_%')] = log_returns[ticker].round(2)
                
                # 2. Rate of Change (ROC) - 36-day, lagged by 1 week
                # Calculate daily ROC using the full extended dataset
                roc_36d = ticker_daily_close.pct_change(periods=36) * 100
                # Convert to weekly (take Friday values) using the FULL dataset including lookback
                roc_weekly_full = roc_36d.resample('W-FRI').last()
                
                # Now manually lag by getting previous week's values for each analysis week
                roc_lagged_values = []
                for current_week in analysis_close.index:
                    # Find the previous week in the full dataset
                    prev_week_candidates = roc_weekly_full.index[roc_weekly_full.index < current_week]
                    if len(prev_week_candidates) > 0:
                        prev_week = prev_week_candidates[-1]  # Most recent previous week
                        roc_lagged_values.append(roc_weekly_full.loc[prev_week])
                    else:
                        roc_lagged_values.append(np.nan)
                
                weekly_data[(ticker, 'ROC_36d_lag1')] = pd.Series(roc_lagged_values, index=analysis_close.index).round(2)
                
                # 3. Relative Volume (RVOL) - 50-day SMA, lagged by 1 week
                # Calculate daily volume SMA using the full extended dataset
                volume_sma_50d = ticker_daily_volume.rolling(window=50).mean()
                rvol_daily = ticker_daily_volume / volume_sma_50d
                # Convert to weekly using the FULL dataset including lookback
                rvol_weekly_full = rvol_daily.resample('W-FRI').last()
                
                # Manually lag by getting previous week's values
                rvol_lagged_values = []
                for current_week in analysis_close.index:
                    prev_week_candidates = rvol_weekly_full.index[rvol_weekly_full.index < current_week]
                    if len(prev_week_candidates) > 0:
                        prev_week = prev_week_candidates[-1]
                        rvol_lagged_values.append(rvol_weekly_full.loc[prev_week])
                    else:
                        rvol_lagged_values.append(np.nan)
                
                weekly_data[(ticker, 'RVOL_50d_lag1')] = pd.Series(rvol_lagged_values, index=analysis_close.index).round(2)
                
                # 4. Bollinger Band Width (BBW) - 36-day, 2 std dev, lagged by 1 week
                # Calculate daily Bollinger Bands using the full extended dataset
                sma_36d = ticker_daily_close.rolling(window=36).mean()
                std_36d = ticker_daily_close.rolling(window=36).std()
                upper_bb = sma_36d + (2 * std_36d)
                lower_bb = sma_36d - (2 * std_36d)
                bbw_daily = ((upper_bb - lower_bb) / sma_36d) * 100
                # Convert to weekly using the FULL dataset including lookback
                bbw_weekly_full = bbw_daily.resample('W-FRI').last()
                
                # Manually lag by getting previous week's values
                bbw_lagged_values = []
                for current_week in analysis_close.index:
                    prev_week_candidates = bbw_weekly_full.index[bbw_weekly_full.index < current_week]
                    if len(prev_week_candidates) > 0:
                        prev_week = prev_week_candidates[-1]
                        bbw_lagged_values.append(bbw_weekly_full.loc[prev_week])
                    else:
                        bbw_lagged_values.append(np.nan)
                
                weekly_data[(ticker, 'BBW_36d_lag1')] = pd.Series(bbw_lagged_values, index=analysis_close.index).round(2)
            
            else:
                # Fill with NaN if insufficient data
                weekly_data[(ticker, 'Log_Return_%')] = np.nan
                weekly_data[(ticker, 'ROC_36d_lag1')] = np.nan
                weekly_data[(ticker, 'RVOL_50d_lag1')] = np.nan
                weekly_data[(ticker, 'BBW_36d_lag1')] = np.nan
    
    # Add week numbers as a separate column
    weekly_data.insert(0, 'Week', range(1, len(weekly_data) + 1))
    
    print(f"📊 Technical Analysis with Optimized Lagged Indicators Complete!")
    print(f"Extended lookback period: {extended_start.date()} to {analysis_start.date()}")
    print(f"Total weeks: {len(weekly_data)}")
    print(f"Date range: {weekly_data.index[0].date()} to {weekly_data.index[-1].date()}")
    print(f"DataFrame shape: {weekly_data.shape}")
    print(f"Stocks analyzed: {len(stocks)}")
    print(f"\n🔍 Calculated Daily-Based Indicators (All Lagged by 1 Week):")
    print("• Log_Return_%: Weekly log returns (current week)")
    print("• ROC_36d_lag1: 36-day Rate of Change (lagged 1 week)")
    print("• RVOL_50d_lag1: Relative Volume vs 50-day SMA (lagged 1 week)")
    print("• BBW_36d_lag1: 36-day Bollinger Band Width (lagged 1 week)")
    
    print(f"\n🎯 Updated Regression Format:")
    print("Return_{i,t} = α + β_MOM×ROC_36d_{i,t-1} + β_BBW×BBW_36d_{i,t-1} + β_VOL×RVOL_50d_{i,t-1} + ε_{i,t}")
    print("\n✅ Improvement: No NaN values in first week due to optimized lagging!")
    print("• Extended lookback provides sufficient data for all calculations")
    print("• Manual lagging preserves all available indicator values")
    print("• Maximum data utilization for robust analysis")
    
    # Check for NaN values in the first few rows
    nan_count_week1 = weekly_data.iloc[0].isna().sum()
    print(f"\n📊 First week NaN check: {nan_count_week1} NaN values out of {len(weekly_data.columns)} total columns")
    
    display(weekly_data.head(10))
        
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

[*********************100%***********************]  14 of 14 completed



Extended lookback period: 2020-09-04 to 2021-01-04
Total weeks: 208
Date range: 2021-01-08 to 2024-12-27
DataFrame shape: (208, 57)
Stocks analyzed: 14

🔍 Calculated Daily-Based Indicators (All Lagged by 1 Week):
• Log_Return_%: Weekly log returns (current week)
• ROC_36d_lag1: 36-day Rate of Change (lagged 1 week)
• RVOL_50d_lag1: Relative Volume vs 50-day SMA (lagged 1 week)
• BBW_36d_lag1: 36-day Bollinger Band Width (lagged 1 week)

📅 Lookback data available: ~44 trading days before analysis period

🎯 Updated Regression Format:
Return_{i,t} = α + β_MOM×ROC_36d_{i,t-1} + β_BBW×BBW_36d_{i,t-1} + β_VOL×RVOL_50d_{i,t-1} + ε_{i,t}
Note: First row will have NaN for lagged indicators due to lag structure

💡 Benefits of Daily Parameters:
• More responsive to market changes (36d ≈ 1.8 months)
• Standard financial industry practice
• Better signal quality with reduced lag


Ticker     Week         AAPL                                          \
Metric          Log_Return_% ROC_36d_lag1 RVOL_50d_lag1 BBW_36d_lag1   
Date                                                                   
2021-01-08    1        -1.11          NaN           NaN          NaN   
2021-01-15    2        -1.60         9.76          0.95        19.68   
2021-01-22    3         8.47        11.67          1.05        16.88   
2021-01-29    4        -8.08        16.82          1.10        14.35   
2021-02-05    5         2.38         6.64          1.65        16.90   
2021-02-12    6        -0.49        12.47          0.70        13.77   
2021-02-19    7        -4.24         5.72          0.58        12.58   
2021-02-26    8        -5.42        -4.85          0.83        12.73   
2021-03-05    9        -1.90        -7.30          1.50        16.46   
2021-03-12   10         0.08        -5.59          1.40        19.19   

Ticker             AMZN                                                 AVGO  \
Metric     Log_Return_% ROC_36d_lag1 RVOL_50d_lag1 BBW_36d_lag1 Log_Return_%   
Date                                                                           
2021-01-08        -2.71          NaN           NaN          NaN         1.43   
2021-01-15        -1.40         1.65          0.85         7.02         1.20   
2021-01-22         5.79         0.19          1.10         6.75         3.67   
2021-01-29        -3.75         3.92          0.76         7.70        -4.15   
2021-02-05         3.33         1.53          1.19         8.42         2.19   
2021-02-12        -2.44         6.18          0.96         9.45         3.67   
2021-02-19        -0.13         2.23          0.64         9.66        -0.18   
2021-02-26        -3.66        -1.04          1.16         9.62        -2.95   
2021-03-05        -4.16        -3.90          1.14        10.87        -6.37   
2021-03-12         2.44        -3.86          1.44        13.59         0.99   

Ticker      ...          TSLA                         V               \
Metric      ... RVOL_50d_lag1 BBW_36d_lag1 Log_Return_% ROC_36d_lag1   
Date        ...                                                        
2021-01-08  ...           NaN          NaN        -2.20          NaN   
2021-01-15  ...          1.62        56.95        -5.95         1.29   
2021-01-22  ...          0.82        55.22        -0.51        -3.15   
2021-01-29  ...          0.43        55.47        -3.60        -3.96   
2021-02-05  ...          0.73        52.52         6.75        -9.12   
2021-02-12  ...          0.42        46.65         0.14         0.73   
2021-02-19  ...          0.57        37.77        -2.86         0.61   
2021-02-26  ...          0.48        26.74         4.25        -3.56   
2021-03-05  ...          1.08        27.24         0.20        -0.83   
2021-03-12  ...          2.60        39.82         3.35         3.30   

Ticker                                         WMT                             \
Metric     RVOL_50d_lag1 BBW_36d_lag1 Log_Return_% ROC_36d_lag1 RVOL_50d_lag1   
Date                                                                            
2021-01-08           NaN          NaN         1.61          NaN           NaN   
2021-01-15          0.77         6.78        -0.93        -3.46          1.11   
2021-01-22          1.07         7.50         1.17        -3.81          1.57   
2021-01-29          0.90         8.69        -3.41        -3.87          0.84   
2021-02-05          2.44        11.98         2.43        -4.80          1.40   
2021-02-12          1.10        12.56        -0.24        -0.88          0.90   
2021-02-19          0.73        12.51        -5.34        -1.03          0.60   
2021-02-26          1.57        12.41        -5.84        -4.74          1.50   
2021-03-05          1.21        11.46        -1.89       -10.86          1.63   
2021-03-12          0.79        12.19         3.63       -13.34          1.28   

Ticker                   
Metri

Getting Technical Indicators for Stocks

In [9]:
# Download data for technical indicator calculations using ta library
try:
    # Need more data to calculate technical indicators properly
    extended_start = dt.date(2020, 1, 1)  # Start earlier for indicator calculations
    
    tech_download = yf.download(
        tickers = stocks,
        start = extended_start,
        end = dt.date(2021, 1, 15),  # Just past first week
        actions = False, threads = True, auto_adjust = True, rounding = True,
        group_by = 'tickers', 
        interval = '1d'  # Daily data for better indicator calculations
    )
    
    # Extract price data
    close_prices = tech_download.xs('Close', level=1, axis=1)
    high_prices = tech_download.xs('High', level=1, axis=1)
    low_prices = tech_download.xs('Low', level=1, axis=1)
    volume_data = tech_download.xs('Volume', level=1, axis=1)
    
    # Find the first week of our analysis period (Jan 4-8, 2021)
    first_week_start = pd.Timestamp('2021-01-04')
    first_week_end = pd.Timestamp('2021-01-08')
    
    # Filter to first week
    first_week_mask = (close_prices.index >= first_week_start) & (close_prices.index <= first_week_end)
    first_week_close = close_prices[first_week_mask]
    
    # Create DataFrame with stocks as rows, indicators as columns
    first_week_indicators = pd.DataFrame(index=stocks, columns=['Log_Return_%', 'ROC', 'RVOL', 'BBW'])
    
    for ticker in stocks:
        # Get price data up to first week end for calculations
        ticker_data = pd.DataFrame({
            'close': close_prices.loc[:first_week_end, ticker],
            'high': high_prices.loc[:first_week_end, ticker],
            'low': low_prices.loc[:first_week_end, ticker],
            'volume': volume_data.loc[:first_week_end, ticker]
        }).dropna()
        
        if len(ticker_data) < 20:  # Need enough data for indicators
            first_week_indicators.loc[ticker] = [np.nan, np.nan, np.nan, np.nan]
            continue
            
        # Log Return % for first week (manual calculation)
        week_open = first_week_close[ticker].iloc[0] if len(first_week_close) > 0 else np.nan
        week_close = first_week_close[ticker].iloc[-1] if len(first_week_close) > 0 else np.nan
        log_return = np.log(week_close / week_open) * 100 if not pd.isna(week_open) and not pd.isna(week_close) else np.nan
        
        # Calculate indicators using ta library (with correct class names)
        # Rate of Change (ROC) - 10-day
        roc = ta.momentum.ROCIndicator(close=ticker_data['close'], window=10).roc().iloc[-1] * 100
        
        # Relative Volume (RVOL) - Current week volume vs 20-day SMA volume
        # Use simple rolling mean since ta library doesn't have VolumeSMAIndicator
        volume_sma_20 = ticker_data['volume'].rolling(window=20).mean()
        avg_volume = volume_sma_20.iloc[-1] if not volume_sma_20.empty else np.nan
        current_week_volume = ticker_data['volume'].iloc[-5:].mean()  # Last 5 days average
        rvol = (current_week_volume / avg_volume) if not pd.isna(avg_volume) and avg_volume != 0 else np.nan
        
        # Bollinger Band Width (BBW) using ta library
        bb_indicator = ta.volatility.BollingerBands(close=ticker_data['close'], window=20, window_dev=2)
        upper = bb_indicator.bollinger_hband().iloc[-1]
        lower = bb_indicator.bollinger_lband().iloc[-1]
        middle = bb_indicator.bollinger_mavg().iloc[-1]
        bbw = ((upper - lower) / middle) * 100 if not pd.isna(upper) and not pd.isna(lower) and middle != 0 else np.nan
            
        # Add row to DataFrame
        first_week_indicators.loc[ticker] = [log_return, roc, rvol, bbw]
    
    print(f"📊 First Week Technical Analysis (Using ta library)")
    print(f"Analysis Period: {first_week_start.date()} to {first_week_end.date()}")
    print(f"Stocks Analyzed: {len(stocks)}")
    print(f"\n🔍 Technical Indicators for First Week:")
    print("• Log_Return_%: Weekly log return")
    print("• ROC: 10-day Rate of Change (ta.momentum)")
    print("• RVOL: Relative Volume vs 20-day average (pandas rolling)")
    print("• BBW: Bollinger Band Width (ta.volatility)")
    
    display(first_week_indicators.round(4))
    
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

[*********************100%***********************]  14 of 14 completed

📊 First Week Technical Analysis (Using ta library)
Analysis Period: 2021-01-04 to 2021-01-08
Stocks Analyzed: 14

🔍 Technical Indicators for First Week:
• Log_Return_%: Weekly log return
• ROC: 10-day Rate of Change (ta.momentum)
• RVOL: Relative Volume vs 20-day average (pandas rolling)
• BBW: Bollinger Band Width (ta.volatility)


,Log_Return_%,ROC,RVOL,BBW
AAPL,2.019636,83.150298,1.073922,12.090838
AMZN,-0.125604,-8.162753,1.022023,6.977552
AVGO,4.688757,474.576271,1.001516,10.537177
GOOG,4.477595,432.4073,1.24789,5.04911
GOOGL,4.069133,403.169424,1.202613,4.774844
JPM,8.468695,953.117888,1.272108,16.987683
LLY,0.545271,55.954728,0.910911,8.616199
META,-0.51011,-20.264945,0.993752,5.801513
MSFT,0.884877,-63.067727,1.058476,7.507613
NVDA,1.21582,208.172706,1.624698,6.065621


In [ ]:
# Goal Code

## 🔬 Multiple Linear Regression Analysis

### Factor Definition & Model Specification

**Our Three-Factor Model:**
```
Return_{i,t} = α + β_MOM × MOM_{i,t-1} + β_BBW × BBW_{i,t-1} + β_VOL × VOL_{i,t-1} + ε_{i,t}
```

### Factor Definitions:

1. **Momentum Factor (MOM_Factor)**: 
   - **36-week Rate of Change (ROC)** 
   - Measures intermediate-term price trends
   - Formula: `ROC = ((Close_t - Close_t-36) / Close_t-36) × 100`

2. **Volatility Factor (BBW_Factor)**: 
   - **36-week Bollinger Band Width (BBW)**
   - Measures recent price volatility using 2 standard deviations
   - Formula: `BBW = ((Upper_Band - Lower_Band) / Moving_Average) × 100`

3. **Volume Factor (VOL_Factor)**: 
   - **50-week Relative Volume** 
   - Compares recent volume to long-term average
   - Formula: `RVOL = Current_Volume / 50_week_Average_Volume`

### Methodology:
- **Cross-sectional regressions** run weekly across our stock universe
- **One-week lag** on all factors to avoid look-ahead bias
- **Weekly time series** from 2021-2024 (4 years)
- **Minimum 10 stocks** required for stable regression results

---

In [78]:
# Step 1: Import required libraries for regression analysis
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

print("🔬 Multiple Linear Regression: Factor Analysis Setup")
print("=" * 55)
print("📊 Analyzing Momentum, Volatility, and Volume Factors")
print("🎯 Model: Return_{i,t} = α + β_MOM×MOM_{i,t-1} + β_BBW×BBW_{i,t-1} + β_VOL×VOL_{i,t-1} + ε_{i,t}")
print("=" * 55)

🔬 Multiple Linear Regression: Factor Analysis Setup
📊 Analyzing Momentum, Volatility, and Volume Factors
🎯 Model: Return_{i,t} = α + β_MOM×MOM_{i,t-1} + β_BBW×BBW_{i,t-1} + β_VOL×VOL_{i,t-1} + ε_{i,t}


In [79]:
# Step 2: Download Extended Dataset for Factor Calculations
print("\n📥 Downloading Extended Dataset for Factor Calculations...")

try:
    # Extended time range - start earlier for proper factor calculations
    extended_start_date = dt.date(2020, 6, 1)  # 6 months before analysis start
    
    print(f"📅 Extended date range: {extended_start_date} to {endDate}")
    print(f"💾 Downloading daily data for {len(stocks)} stocks...")
    
    # Download daily data for accurate factor calculations
    extended_data = yf.download(
        tickers=stocks,
        start=extended_start_date,
        end=endDate,
        actions=False, 
        threads=True, 
        auto_adjust=True, 
        rounding=True,
        group_by='tickers', 
        interval='1d'
    )
    
    # Extract OHLCV data for each component
    daily_close = extended_data.xs('Close', level=1, axis=1)
    daily_high = extended_data.xs('High', level=1, axis=1)
    daily_low = extended_data.xs('Low', level=1, axis=1)
    daily_volume = extended_data.xs('Volume', level=1, axis=1)
    daily_open = extended_data.xs('Open', level=1, axis=1)
    
    # Convert daily to weekly data (Friday close)
    weekly_close_ext = daily_close.resample('W-FRI').last()
    weekly_high_ext = daily_high.resample('W-FRI').max()
    weekly_low_ext = daily_low.resample('W-FRI').min() 
    weekly_volume_ext = daily_volume.resample('W-FRI').sum()
    weekly_open_ext = daily_close.resample('W-FRI').first()
    
    # Filter to our main analysis period (2021 onwards)
    analysis_start_date = pd.Timestamp('2021-01-01')
    mask = weekly_close_ext.index >= analysis_start_date
    
    weekly_close_analysis = weekly_close_ext[mask]
    weekly_high_analysis = weekly_high_ext[mask]
    weekly_low_analysis = weekly_low_ext[mask]
    weekly_volume_analysis = weekly_volume_ext[mask]
    weekly_open_analysis = weekly_open_ext[mask]
    
    print(f"✅ Data download successful!")
    print(f"📊 Analysis period: {weekly_close_analysis.index[0].date()} to {weekly_close_analysis.index[-1].date()}")
    print(f"📈 Total weeks in analysis: {len(weekly_close_analysis)}")
    print(f"🏢 Stocks: {len(stocks)}")
    print(f"📋 Data shape: {weekly_close_analysis.shape}")
    
except Exception as e:
    print(f"❌ Error downloading extended data: {e}")
    raise


📥 Downloading Extended Dataset for Factor Calculations...
📅 Extended date range: 2020-06-01 to 2024-12-27
💾 Downloading daily data for 14 stocks...


[*********************100%***********************]  14 of 14 completed

✅ Data download successful!
📊 Analysis period: 2021-01-01 to 2024-12-27
📈 Total weeks in analysis: 209
🏢 Stocks: 14
📋 Data shape: (209, 14)


In [80]:
# Step 3: Calculate Three Factor Exposures
print("\n📊 Calculating Factor Exposures for Each Stock...")
print("=" * 50)

# Initialize storage for factors and returns
factor_df = pd.DataFrame(index=weekly_close_analysis.index)

# Process each stock to calculate factors
successful_stocks = []

for ticker in stocks:
    try:
        print(f"🔄 Processing {ticker}...", end="")
        
        # Get weekly price and volume data for this stock
        weekly_prices = pd.DataFrame({
            'close': weekly_close_ext[ticker],
            'high': weekly_high_ext[ticker],
            'low': weekly_low_ext[ticker],
            'volume': weekly_volume_ext[ticker],
            'open': weekly_open_ext[ticker]
        }).dropna()
        
        # Ensure we have enough data for calculations
        if len(weekly_prices) < 60:  # Need 50+ weeks for volume factor
            print(" ⚠️ Insufficient data")
            continue
            
        # FACTOR 1: MOMENTUM (36-week Rate of Change)
        # ROC = ((Close_t - Close_t-36) / Close_t-36) * 100
        momentum_36w = weekly_prices['close'].pct_change(periods=36) * 100
        
        # FACTOR 2: VOLATILITY (36-week Bollinger Band Width) 
        # Calculate 36-week moving average and standard deviation
        sma_36w = weekly_prices['close'].rolling(window=36).mean()
        std_36w = weekly_prices['close'].rolling(window=36).std()
        
        # Bollinger Bands (±2 standard deviations)
        upper_bb = sma_36w + (2 * std_36w)
        lower_bb = sma_36w - (2 * std_36w)
        
        # Bollinger Band Width = ((Upper - Lower) / SMA) * 100
        volatility_36w = ((upper_bb - lower_bb) / sma_36w) * 100
        
        # FACTOR 3: VOLUME (50-week Relative Volume)
        # RVOL = Current_Volume / 50_week_Average_Volume
        vol_sma_50w = weekly_prices['volume'].rolling(window=50).mean()
        volume_50w = weekly_prices['volume'] / vol_sma_50w
        
        # DEPENDENT VARIABLE: Weekly Log Returns
        weekly_log_returns = np.log(weekly_prices['close'] / weekly_prices['open']) * 100
        
        # Filter to analysis period and store
        analysis_mask = weekly_prices.index >= analysis_start_date
        
        factor_df[f'{ticker}_MOM'] = momentum_36w[analysis_mask]
        factor_df[f'{ticker}_VOL'] = volatility_36w[analysis_mask]
        factor_df[f'{ticker}_RVOL'] = volume_50w[analysis_mask]
        factor_df[f'{ticker}_RETURN'] = weekly_log_returns[analysis_mask]
        
        successful_stocks.append(ticker)
        print(" ✅")
        
    except Exception as e:
        print(f" ❌ Error: {e}")
        continue

print(f"\n✅ Factor Calculation Complete!")
print(f"📈 Successfully processed: {len(successful_stocks)} stocks")
print(f"📊 Factor data shape: {factor_df.shape}")

# Remove rows with too many NaN values
factor_df_clean = factor_df.dropna(thresh=len(successful_stocks)*2)  # At least 50% valid data
print(f"📋 Clean data shape: {factor_df_clean.shape}")
print(f"📅 Analysis period: {factor_df_clean.index[0].date()} to {factor_df_clean.index[-1].date()}")

# Display sample of factor data
print(f"\n📋 Sample Factor Data (First 5 weeks, First 2 stocks):")
sample_cols = []
for ticker in successful_stocks[:2]:
    sample_cols.extend([f'{ticker}_RETURN', f'{ticker}_MOM', f'{ticker}_VOL', f'{ticker}_RVOL'])

display(factor_df_clean[sample_cols].head().round(3))


📊 Calculating Factor Exposures for Each Stock...
🔄 Processing AAPL... ✅
🔄 Processing AMZN... ✅
🔄 Processing AVGO... ✅
🔄 Processing GOOG... ✅
🔄 Processing GOOGL... ✅
🔄 Processing JPM... ✅
🔄 Processing LLY... ✅
🔄 Processing META... ✅
🔄 Processing MSFT... ✅
🔄 Processing NVDA... ✅
🔄 Processing ORCL... ✅
🔄 Processing TSLA... ✅
🔄 Processing V... ✅
🔄 Processing WMT... ✅

✅ Factor Calculation Complete!
📈 Successfully processed: 14 stocks
📊 Factor data shape: (209, 56)
📋 Clean data shape: (204, 56)
📅 Analysis period: 2021-02-05 to 2024-12-27

📋 Sample Factor Data (First 5 weeks, First 2 stocks):


,AAPL_RETURN,AAPL_MOM,AAPL_VOL,AAPL_RVOL,AMZN_RETURN,AMZN_MOM,AMZN_VOL,AMZN_RVOL
Date,,,,,,,,
2021-02-05,2.082,NaN,54.500,NaN,0.281,NaN,27.617,NaN
2021-02-12,-1.129,64.161,51.934,NaN,-1.370,32.010,24.014,NaN
2021-02-19,-2.524,54.098,48.716,NaN,-0.589,27.694,20.334,NaN
2021-02-26,-3.838,39.392,45.234,NaN,-2.799,15.626,17.535,NaN
2021-03-05,-5.108,38.036,41.574,NaN,-4.745,11.423,14.736,NaN


In [ ]:
# Step 4: Weekly Cross-Sectional Multiple Linear Regression
print("\n🔬 Running Weekly Cross-Sectional Multiple Linear Regressions...")
print("=" * 60)
print("🎯 Model: Return_{i,t} = α + β_MOM×MOM_{i,t-1} + β_VOL×VOL_{i,t-1} + β_RVOL×RVOL_{i,t-1} + ε_{i,t}")
print("📅 Using 1-week lag to avoid look-ahead bias")

# Storage for regression results
regression_results = []

# Get clean weeks for regression (skip first week due to lagging)
regression_weeks = factor_df_clean.index[1:]  # Start from week 2

print(f"📊 Running regressions for {len(regression_weeks)} weeks...")

for i, current_week in enumerate(regression_weeks):
    try:
        # Get previous week index for lagged factors
        prev_week_idx = factor_df_clean.index.get_loc(current_week) - 1
        previous_week = factor_df_clean.index[prev_week_idx]
        
        # Collect cross-sectional data for this week
        weekly_returns = []  # Dependent variable: Returns at time t
        weekly_factors = []  # Independent variables: Factors at time t-1 (lagged)
        stock_names = []
        
        for ticker in successful_stocks:
            # Current week return (dependent variable)
            return_col = f'{ticker}_RETURN'
            # Previous week factors (independent variables) 
            mom_col = f'{ticker}_MOM'
            vol_col = f'{ticker}_VOL'
            rvol_col = f'{ticker}_RVOL'
            
            # Get the data
            if all(col in factor_df_clean.columns for col in [return_col, mom_col, vol_col, rvol_col]):
                current_return = factor_df_clean.loc[current_week, return_col]
                lagged_mom = factor_df_clean.loc[previous_week, mom_col]
                lagged_vol = factor_df_clean.loc[previous_week, vol_col] 
                lagged_rvol = factor_df_clean.loc[previous_week, rvol_col]
                
                # Only include if all values are valid (not NaN or infinite)
                values = [current_return, lagged_mom, lagged_vol, lagged_rvol]
                if all(pd.notna(values)) and all(np.isfinite(values)):
                    weekly_returns.append(current_return)
                    weekly_factors.append([lagged_mom, lagged_vol, lagged_rvol])
                    stock_names.append(ticker)
        
        # Run regression if we have sufficient observations
        min_stocks = 8  # Minimum stocks needed for stable regression
        if len(weekly_returns) >= min_stocks:
            
            # Convert to numpy arrays
            y = np.array(weekly_returns)
            X = np.array(weekly_factors)
            
            # Fit regression using scikit-learn
            reg_model = LinearRegression()
            reg_model.fit(X, y)
            
            # Calculate predictions and residuals
            y_pred = reg_model.predict(X)
            residuals = y - y_pred
            
            # Calculate R-squared
            r_squared = r2_score(y, y_pred)
            
            # Calculate adjusted R-squared  
            n = len(y)  # number of observations
            k = X.shape[1]  # number of features
            adj_r_squared = 1 - (1 - r_squared) * (n - 1) / (n - k - 1)
            
            # Calculate F-statistic for overall model significance
            mse_model = np.sum((y_pred - np.mean(y))**2) / k
            mse_residual = np.sum(residuals**2) / (n - k - 1)
            f_statistic = mse_model / mse_residual if mse_residual > 0 else 0
            f_pvalue = 1 - stats.f.cdf(f_statistic, k, n - k - 1) if f_statistic > 0 else 1
            
            # Calculate t-statistics and p-values for coefficients
            # Standard errors calculation
            X_with_intercept = np.column_stack([np.ones(len(X)), X])
            try:
                # Calculate standard errors using matrix algebra
                XtX_inv = np.linalg.inv(X_with_intercept.T @ X_with_intercept)
                mse = np.sum(residuals**2) / (n - k - 1)
                var_coeff = mse * np.diag(XtX_inv)
                std_errors = np.sqrt(var_coeff)
                
                # Coefficients (intercept + slopes)
                all_coeffs = np.array([reg_model.intercept_] + list(reg_model.coef_))
                
                # T-statistics
                t_stats = all_coeffs / std_errors
                
                # P-values (two-tailed test)
                p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), n - k - 1))
                
            except np.linalg.LinAlgError:
                # If matrix inversion fails, set default values
                std_errors = np.ones(k + 1)
                t_stats = np.zeros(k + 1)
                p_values = np.ones(k + 1)
                all_coeffs = np.array([reg_model.intercept_] + list(reg_model.coef_))
            
            # Store comprehensive results
            result_dict = {
                'Week_Number': i + 2,  # Actual week number 
                'Date': current_week.date(),
                'N_Stocks': len(weekly_returns),
                'Alpha': reg_model.intercept_,  # Intercept
                'Beta_MOM': reg_model.coef_[0],  # Momentum coefficient
                'Beta_VOL': reg_model.coef_[1],  # Volatility coefficient  
                'Beta_RVOL': reg_model.coef_[2],  # Relative Volume coefficient
                'R_squared': r_squared,
                'Adj_R_squared': adj_r_squared,
                'F_statistic': f_statistic,
                'F_pvalue': f_pvalue,
                'Alpha_pvalue': p_values[0],
                'Beta_MOM_pvalue': p_values[1], 
                'Beta_VOL_pvalue': p_values[2],
                'Beta_RVOL_pvalue': p_values[3],
                'Alpha_tstat': t_stats[0],
                'Beta_MOM_tstat': t_stats[1],
                'Beta_VOL_tstat': t_stats[2], 
                'Beta_RVOL_tstat': t_stats[3],
                'Mean_Return': np.mean(y),
                'Std_Return': np.std(y)
            }
            
            regression_results.append(result_dict)
            
            # Progress indicator
            if (i + 1) % 20 == 0:
                print(f"📊 Completed {i + 1} regressions...")
                    
        else:
            if (i + 1) % 50 == 0:  # Less frequent warning for insufficient stocks
                print(f"⚠️ Week {i+2}: Only {len(weekly_returns)} stocks available (minimum {min_stocks} required)")
            
    except Exception as e:
        print(f"❌ Error processing week {i+2}: {e}")
        continue

# Convert results to DataFrame for analysis
mlr_results_df = pd.DataFrame(regression_results)

print(f"\n✅ Multiple Linear Regression Analysis Complete!")
print(f"📊 Successfully analyzed {len(mlr_results_df)} weeks")
if len(mlr_results_df) > 0:
    print(f"📈 Average stocks per regression: {mlr_results_df['N_Stocks'].mean():.1f}")
    print(f"📅 Period coverage: {mlr_results_df['Date'].min()} to {mlr_results_df['Date'].max()}") 
else:
    print("⚠️ No successful regressions completed")

# Store results globally for further analysis  
globals()['mlr_results'] = mlr_results_df
globals()['factor_data_final'] = factor_df_clean

print(f"\n💾 Results stored in variables:")
print(f"   • 'mlr_results': Weekly regression results")
print(f"   • 'factor_data_final': Complete factor dataset")


🔬 Running Weekly Cross-Sectional Multiple Linear Regressions...
🎯 Model: Return_{i,t} = α + β_MOM×MOM_{i,t-1} + β_VOL×VOL_{i,t-1} + β_RVOL×RVOL_{i,t-1} + ε_{i,t}
📅 Using 1-week lag to avoid look-ahead bias
📊 Running regressions for 203 weeks...
⚠️ Week 2: Only 0 stocks available (minimum 8 required)
⚠️ Week 3: Only 0 stocks available (minimum 8 required)
⚠️ Week 4: Only 0 stocks available (minimum 8 required)
⚠️ Week 5: Only 0 stocks available (minimum 8 required)
⚠️ Week 6: Only 0 stocks available (minimum 8 required)
⚠️ Week 7: Only 0 stocks available (minimum 8 required)
⚠️ Week 8: Only 0 stocks available (minimum 8 required)
⚠️ Week 9: Only 0 stocks available (minimum 8 required)
⚠️ Week 10: Only 0 stocks available (minimum 8 required)
⚠️ Week 11: Only 0 stocks available (minimum 8 required)
⚠️ Week 12: Only 0 stocks available (minimum 8 required)
⚠️ Week 13: Only 0 stocks available (minimum 8 required)
⚠️ Week 14: Only 0 stocks available (minimum 8 required)
⚠️ Week 15: Only 0 st

KeyError: 'N_Stocks'

In [76]:
# Step 5: Comprehensive Results Analysis & Summary
print("\n📈 MULTIPLE LINEAR REGRESSION RESULTS SUMMARY")
print("=" * 60)

if len(mlr_results_df) > 0:
    # Model Performance Metrics
    print(f"🔸 OVERALL MODEL PERFORMANCE")
    print(f"   Total weeks analyzed: {len(mlr_results_df)}")
    print(f"   Average R-squared: {mlr_results_df['R_squared'].mean():.4f}")
    print(f"   Average Adjusted R-squared: {mlr_results_df['Adj_R_squared'].mean():.4f}")
    print(f"   Model significance (F-test p<0.05): {(mlr_results_df['F_pvalue'] < 0.05).sum()}/{len(mlr_results_df)} weeks ({(mlr_results_df['F_pvalue'] < 0.05).mean()*100:.1f}%)")
    
    # Factor Significance Analysis
    print(f"\n🔸 FACTOR SIGNIFICANCE ANALYSIS (p < 0.05)")
    
    # Calculate significance statistics
    sig_alpha = (mlr_results_df['Alpha_pvalue'] < 0.05).sum()
    sig_mom = (mlr_results_df['Beta_MOM_pvalue'] < 0.05).sum() 
    sig_vol = (mlr_results_df['Beta_VOL_pvalue'] < 0.05).sum()
    sig_rvol = (mlr_results_df['Beta_RVOL_pvalue'] < 0.05).sum()
    total_weeks = len(mlr_results_df)
    
    print(f"   📊 Alpha (Intercept):")
    print(f"      • Average coefficient: {mlr_results_df['Alpha'].mean():.4f}")
    print(f"      • Significant weeks: {sig_alpha}/{total_weeks} ({sig_alpha/total_weeks*100:.1f}%)")
    print(f"      • Average t-statistic: {mlr_results_df['Alpha_tstat'].mean():.2f}")
    
    print(f"   📈 Momentum Factor (β_MOM - 36w ROC):")
    print(f"      • Average coefficient: {mlr_results_df['Beta_MOM'].mean():.4f}")
    print(f"      • Significant weeks: {sig_mom}/{total_weeks} ({sig_mom/total_weeks*100:.1f}%)")
    print(f"      • Average t-statistic: {mlr_results_df['Beta_MOM_tstat'].mean():.2f}")
    
    print(f"   📊 Volatility Factor (β_VOL - 36w BBW):")
    print(f"      • Average coefficient: {mlr_results_df['Beta_VOL'].mean():.4f}")
    print(f"      • Significant weeks: {sig_vol}/{total_weeks} ({sig_vol/total_weeks*100:.1f}%)")
    print(f"      • Average t-statistic: {mlr_results_df['Beta_VOL_tstat'].mean():.2f}")
    
    print(f"   📈 Volume Factor (β_RVOL - 50w Relative Volume):")
    print(f"      • Average coefficient: {mlr_results_df['Beta_RVOL'].mean():.4f}")
    print(f"      • Significant weeks: {sig_rvol}/{total_weeks} ({sig_rvol/total_weeks*100:.1f}%)")
    print(f"      • Average t-statistic: {mlr_results_df['Beta_RVOL_tstat'].mean():.2f}")
    
    # Economic Interpretation
    print(f"\n🔸 ECONOMIC INTERPRETATION")
    avg_mom = mlr_results_df['Beta_MOM'].mean()
    avg_vol = mlr_results_df['Beta_VOL'].mean() 
    avg_rvol = mlr_results_df['Beta_RVOL'].mean()
    
    if avg_mom > 0:
        print(f"   📈 Momentum: Positive average coefficient ({avg_mom:.4f}) suggests momentum effect")
    else:
        print(f"   📉 Momentum: Negative average coefficient ({avg_mom:.4f}) suggests mean reversion")
        
    if avg_vol > 0:
        print(f"   📊 Volatility: Positive average coefficient ({avg_vol:.4f}) suggests higher volatility predicts higher returns")
    else:
        print(f"   📊 Volatility: Negative average coefficient ({avg_vol:.4f}) suggests higher volatility predicts lower returns")
        
    if avg_rvol > 0:
        print(f"   📈 Volume: Positive average coefficient ({avg_rvol:.4f}) suggests higher volume predicts higher returns")
    else:
        print(f"   📉 Volume: Negative average coefficient ({avg_rvol:.4f}) suggests higher volume predicts lower returns")
    
    # Display sample results
    print(f"\n📋 SAMPLE REGRESSION RESULTS (First 10 weeks)")
    print("=" * 80)
    display_cols = ['Week_Number', 'Date', 'N_Stocks', 'Alpha', 'Beta_MOM', 'Beta_VOL', 'Beta_RVOL', 'R_squared', 'F_pvalue']
    display(mlr_results_df[display_cols].head(10).round(4))
    
    print(f"\n📋 STATISTICAL SUMMARY OF COEFFICIENTS")
    print("=" * 50)
    coeff_summary = mlr_results_df[['Alpha', 'Beta_MOM', 'Beta_VOL', 'Beta_RVOL']].describe()
    display(coeff_summary.round(4))
    
else:
    print("❌ No regression results available. Check data and parameters.")

print(f"\n🎯 MODEL SPECIFICATION SUMMARY")
print("=" * 35)
print("Return_{i,t} = α + β_MOM × MOM_{i,t-1} + β_VOL × VOL_{i,t-1} + β_RVOL × RVOL_{i,t-1} + ε_{i,t}")
print("\nWhere:")
print("• Return_{i,t}: Weekly log return for stock i at time t")
print("• MOM_{i,t-1}: 36-week Rate of Change (momentum), lagged 1 week")
print("• VOL_{i,t-1}: 36-week Bollinger Band Width (volatility), lagged 1 week")
print("• RVOL_{i,t-1}: 50-week Relative Volume (volume), lagged 1 week")
print("• α: Intercept (average unexplained return)")
print("• β: Factor loadings (sensitivity to each factor)")
print("• ε_{i,t}: Residual error term")

print(f"\n✅ Analysis Complete! Results stored for further investigation.")


📈 MULTIPLE LINEAR REGRESSION RESULTS SUMMARY
❌ No regression results available. Check data and parameters.

🎯 MODEL SPECIFICATION SUMMARY
Return_{i,t} = α + β_MOM × MOM_{i,t-1} + β_VOL × VOL_{i,t-1} + β_RVOL × RVOL_{i,t-1} + ε_{i,t}

Where:
• Return_{i,t}: Weekly log return for stock i at time t
• MOM_{i,t-1}: 36-week Rate of Change (momentum), lagged 1 week
• VOL_{i,t-1}: 36-week Bollinger Band Width (volatility), lagged 1 week
• RVOL_{i,t-1}: 50-week Relative Volume (volume), lagged 1 week
• α: Intercept (average unexplained return)
• β: Factor loadings (sensitivity to each factor)
• ε_{i,t}: Residual error term

✅ Analysis Complete! Results stored for further investigation.
